In [11]:
!pip install tweet-preprocessor
!pip install GetOldTweets3
!pip install swifter

In [12]:
!pip install -U ipykernel

Requirement already up-to-date: ipykernel in /usr/local/lib/python3.6/dist-packages (5.3.0)


In [0]:
import swifter
import tweepy
from tweepy import Stream
from tweepy import OAuthHandler
from tweepy.streaming import StreamListener
import json
import matplotlib.pyplot as plt
import pandas as pd
import preprocessor as p
import csv
import string
import re
from wordcloud import WordCloud
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

from textblob import TextBlob
import GetOldTweets3 as got

### Timeline : from Jan 2020 to Present
We need to extract before and after COVID to show the variation

| Analysis Parameter | Description | Value Type |
| --- | --- | --- |
| Age | Victim & Abuser | Number / binned groups |
| Gender | Victim & Abuser | M/F/Others |
| Sexual Orientation | Victim & Abuser | Lesbian,Gay,Bisexual,Transperson,Pansexual,Intersex,Heterosexual,Queer,LGBT,LGBTQ+ |
| Location | Victim & Abuser | City / LatLong data |
| Language | Language of report filed | English,Hindi,Hinglish,Punjabi,Bengali,Telugu,Tamil,Malyalam,Kannada,Assamese |
| Time of Day | When reported & When incident took place | GST/IST |
| Relationship | B/W victim & abuser | family/friend etc |
| Visual Imagery | In report | bool(0/1): if 1, further analysis |
| - | - | - |
| Abuse Type | Clustering : Can be inferred from above features | Physical/Sexual/Psychological/Deprivation |
| User demographic | Clustering : Can be inferred from above features for both victim & abuser | housewife, domestic helper, blue collar, white Collar maybe |


We're trying to find these features using Twitter data.

In [0]:
import pandas as pd
import numpy as np
raw_english_categories = pd.read_excel("Domestic Violence & Online Harassment in COVID-19 .xlsx",sheet_name="CATEGORICAL_ENGLISH")
tamil = pd.read_excel("Domestic Violence & Online Harassment in COVID-19 .xlsx",sheet_name="tamil")

In [0]:
raw_english_categories.head()

,English,category
0,Slap,D
1,pinch,D
2,Thapad,D
3,pull Hair,D
4,Burn,D


In [0]:
domestic_violence_words = raw_english_categories[(raw_english_categories['category'] == "D") | (raw_english_categories['category'] == "B")]
harassment_words = raw_english_categories[(raw_english_categories['category'] == "H") | (raw_english_categories['category'] == "B")]

In [0]:
len(domestic_violence_words["English"].unique())

128

In [0]:
len(harassment_words["English"].unique())

63

In [0]:
if "domestic violence" in domestic_violence_words["English"].unique():
  print("true")

true


In [0]:
domestic_violence = pd.DataFrame()

In [0]:
domestic_violence_words['English'].unique()[22:]

array(['piece of shit ', 'hope you die ', 'little bitch', 'dumb slut',
       'clout ', 'the fuck out of her/him ', 'fuck yourself ',
       'grow a pair/ of balls', 'abuse', 'asshat', 'asskisser',
       'unsolicited ', 'cuntfucker', 'dumbfuck', 'fag', 'faggot',
       'fucktard ', 'mental', 'crazy', 'shame', 'supress', 'intimidate',
       'rude', 'fear ', 'attack', 'victim', 'in-laws', 'assault', 'force',
       'crying', 'alone', 'incident', 'stalk', 'call girl', 'item girl',
       'fucker', 'last warning', 'ego', 'Porn', 'Prostitute', 'Hooker',
       'murder', 'killing', 'alert', 'unsafe', 'pain', 'ravish', 'wank',
       'ass', 'wanking', 'choking', 'choke', 'burn', 'ugly', 'bitch',
       'kitty-Cat', 'punch', 'bite', 'eat shit', 'eat that cock',
       'eat that pussy', 'throwing objects', 'hitting me', 'pushing me',
       'not feeding me', 'financial abuse', 'suffering ', 'torture',
       'isolated me', 'criticize', 'control money', 'victim blaming',
       'hacked my acco

In [0]:
tamil.head()

,Tamil
0,Akkula thukki kami di
1,Amarjith
2,Anaathai Kaluthai!!!!
3,Arivu ketta koothi
4,Avuthu kami


In [0]:
# Central India lat/long : 23, 80
# N-S India length : 3300 kms
tamil_df = pd.DataFrame()
count = 0
for words in tamil['Tamil'].unique():
  print(count)
  count += 1
  tweetCriteria = got.manager.TweetCriteria().setQuerySearch(words.lower())\
                                             .setSince("2020-01-01")\
                                             .setUntil("2020-05-21")\
                                             .setNear("10.8162, 79.6165")\
                                             .setWithin("702km")\
                                             .setMaxTweets(500)

  tweets = got.manager.TweetManager.getTweets(tweetCriteria)
  if len(tweets):
    features = tweets[0].__dict__.keys()
    df = pd.DataFrame([[getattr(i,j) for j in features] for i in tweets], columns = features)
    df.drop(columns=['to', 'permalink', 'formatted_date','urls'],inplace=True)
    df.rename(columns={'id': 'tweet_id','author_id': 'user_id', 'geo':'tweet_location'}, inplace=True)
    tamil_df = tamil_df.append(df)
  else:
    print("No tweets found")
#tamil_df.drop(columns=['to', 'permalink', 'formatted_date','urls'],inplace=True)
#tamil_df.rename(columns={'id': 'tweet_id','author_id': 'user_id', 'geo':'tweet_location'}, inplace=True)

141
142
143
No tweets found
144
No tweets found
145
No tweets found
146
147
No tweets found
148
No tweets found
149
150
No tweets found
151
No tweets found
152
No tweets found
153
No tweets found
154
155
156
157
158
No tweets found
159
No tweets found
160
161
No tweets found
162
163
164
No tweets found
165
No tweets found
166
No tweets found
167
No tweets found
168
No tweets found
169
170
No tweets found
171
No tweets found
172
No tweets found
173
No tweets found
174
175
No tweets found
176
No tweets found
177
No tweets found
178
No tweets found
179
180
No tweets found
181
182
No tweets found
183
184
185
186
No tweets found
187
No tweets found
188
No tweets found
189
No tweets found
190
No tweets found
191
No tweets found
192
No tweets found
193


In [0]:
tamil_df.shape

(3766, 11)

In [0]:
tamil_df.head()

,username,text,retweets,favorites,replies,tweet_id,user_id,date,hashtags,mentions,tweet_location
0,amarjit_kharga,@Google Hi i am unable to recover my gmail id ...,0,0,0,1263184201521590272,3514638193,2020-05-20 19:05:57+00:00,,@Google,
1,amarjit_kharga,"@Google Hi , I am not able to recover my gmail...",0,0,0,1263183223913189376,3514638193,2020-05-20 19:02:04+00:00,,@Google,
2,Amarjith7A,എന്നാ ഒരു ഇതാ #HappyBirthdayMohanlal,1,0,0,1263132239765753857,2857754209,2020-05-20 15:39:28+00:00,#HappyBirthdayMohanlal,,
3,Amarjith7A,Screen Presence എന്ന് കേട്ടിട്ടുണ്ടോ ഡാ സ്റ്റീ...,2,0,0,1263132049080115200,2857754209,2020-05-20 15:38:43+00:00,#HappyBirthdayMohanlal,,
4,Amarjith7A,എന്നാണാവോ ഇൗ വിശ്വരൂപം കാണാൻ കിട്ടുവാ #HappyBi...,3,0,0,1263131620816515073,2857754209,2020-05-20 15:37:00+00:00,#HappyBirthdayMohanlal,,


In [0]:
tamil_df.to_csv("domestic_tn.csv",index=False)

In [0]:
def get_location(username):
  try:
      userObj = api.get_user(username)
      print(userObj.location)
      return userObj.location
  except:
      return "No location"

In [0]:
tamil_df['user_location'] = tamil_df['username'].apply(lambda u: get_location(u))

Bengaluru, India
Bengaluru, India
Kerala, India
Kerala, India
Kerala, India
Kerala, India
Kerala, India
Kerala, India
Kerala, India
Kerala, India
Kerala, India
Bangalore

Mumbai & Chennai
Namakkal, Tamil Nadu
Tamil Nadu, India
Tamil Nadu, India
திண்டுக்கல்🗡
Chennai, India
சென்னை
Coimbatore
Chennai, India
Chennai 
chennai
Chennai, India
திண்டுக்கல்🗡
Chennai, India
Chennai
Tirunelveli, India
Chennai, India
Chennai, India
chennai
India
Ariyalur
Avadi Chennai
Avadi Chennai
chennai
Avadi Chennai
Avadi Chennai
Kerala, India
chennai
Kerala, India
Chennai, India
Chennai, India
Chennai
coimbatore
Chennai, India
Tirunelveli
திண்டுக்கல்🗡
திண்டுக்கல்🗡
Chennai
Chennai, India
Tamil Nadu, India
Chennai, India
Chennai, India
coimbatore
CHENNAI
CHENNAI
Coimbatore, India
CHENNAI
CHENNAI
CHENNAI
Tirunelveli, India
Bengaluru, India
Chennai, India
Pondicherry, India
Chennai, India
Chennai, India
Chennai
Kerala, India
Tirunelveli, India
Kerala, India
Trivandrum, India
Trivandrum, India
India
Chennai, India


In [0]:
tamil_df.to_csv("domestic_tn_states.csv",index=False)
from google.colab import files
files.download("domestic_tn_states.csv")

In [0]:
tamil_df.head(2)

,username,to,text,retweets,favorites,replies,id,permalink,author_id,date,formatted_date,hashtags,mentions,geo,urls,tweet_id,user_id,tweet_location
0,amo_chatha,emilyhewertson,"They are shit mate, how much they helping big ...",0,0,0,1263245800920043520,https://twitter.com/amo_chatha/status/12632458...,459795609.0,2020-05-20 23:10:43+00:00,Wed May 20 23:10:43 +0000 2020,,,,https://twitter.com/emilyhewertson/status/1262...,NaN,NaN,TamilNadu
1,Amarjitmand,TrollFootball,@KezSBops,0,0,1,1263212576554602497,https://twitter.com/Amarjitmand/status/1263212...,287674883.0,2020-05-20 20:58:42+00:00,Wed May 20 20:58:42 +0000 2020,,@KezSBops,,,NaN,NaN,TamilNadu


In [0]:
tamil_df.to_csv("tamil_words.csv",index=False)

In [0]:
tamil_df.shape

(82153, 18)

In [0]:
domestic_violence_words.head()

,English,category
0,Slap,D
1,pinch,D
2,Thapad,D
3,pull Hair,D
4,Burn,D


In [0]:
domestic_violence_words['English'].unique()

array(['Slap', 'pinch', 'Thapad', 'pull Hair', 'Burn', 'Cigarette burn',
       'Tie', 'jerkoff', 'asshole ', 'fuck ', 'bitch ', 'slut ', 'whore ',
       'hoe', 'hell ', 'son of a bitch', 'bastard ', 'fuck off ',
       'motherfucker ', 'bitchass ', 'cocksucker ', 'kiss my ass ',
       'piece of shit ', 'hope you die ', 'little bitch', 'dumb slut',
       'clout ', 'the fuck out of her/him ', 'fuck yourself ',
       'grow a pair/ of balls', 'abuse', 'asshat', 'asskisser',
       'unsolicited ', 'cuntfucker', 'dumbfuck', 'fag', 'faggot',
       'fucktard ', 'mental', 'crazy', 'shame', 'supress', 'intimidate',
       'rude', 'fear ', 'attack', 'victim', 'in-laws', 'assault', 'force',
       'crying', 'alone', 'incident', 'stalk', 'call girl', 'item girl',
       'fucker', 'last warning', 'ego', 'Porn', 'Prostitute', 'Hooker',
       'murder', 'killing', 'alert', 'unsafe', 'pain', 'ravish', 'wank',
       'ass', 'wanking', 'choking', 'choke', 'burn', 'ugly', 'bitch',
       'kitty-Cat'

In [0]:
len(raw_english_categories['English'].unique())

196

In [0]:
len(domestic_violence_words['English'].unique())

128

In [0]:
# Central India lat/long : 23, 80
# N-S India length : 3300 kms
domestic_india = pd.DataFrame()
count = 0
for words in domestic_violence_words['English'].unique():
  tweetCriteria = got.manager.TweetCriteria().setQuerySearch(words.lower())\
                                             .setSince("2020-01-01")\
                                             .setUntil("2020-05-21")\
                                             .setNear("23,80")\
                                             .setWithin("3300km")\
                                             .setMaxTweets(500)

  tweets = got.manager.TweetManager.getTweets(tweetCriteria)
  print(count)
  count += 1
  if len(tweets):
    features = tweets[0].__dict__.keys()
    df = pd.DataFrame([[getattr(i,j) for j in features] for i in tweets], columns = features)
    df.drop(columns=['to', 'permalink', 'formatted_date','urls'],inplace=True)
    df.rename(columns={'id': 'tweet_id','author_id': 'user_id', 'geo':'tweet_location'}, inplace=True)
    domestic_india = domestic_india.append(df)
  else:
    print("No tweets found")


An error occured during an HTTP request: HTTP Error 429: Too Many Requests
Try to open in browser: https://twitter.com/search?q=physical%20near%3A%2223%2C80%22%20within%3A3300km%20since%3A2020-01-01%20until%3A2020-05-21&src=typd


SystemExit: ignored

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [0]:
domestic_india.shape

(51419, 11)

In [0]:
domestic_india.to_csv("domestic_india.csv",index=False)

In [0]:
consumer_key = 'roE8oouLX19kWXsa6oTcwgkY2'
consumer_secret = 'NFcewJQRREqq6CPNyhlwm6Emh2ELvdYeU3M0RJqvgaLnzSTltb'
access_key = '1317466770-Hh9VTreJOVPzwUE0kGsO9RRtlgzpIo6WzzAKWnH'
access_secret = 'AEM9C2pOxli1H0Ca6npnQnwZiIsZqPTL9QhIGHaWJ6qlR'

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_key, access_secret)
api = tweepy.API(auth, wait_on_rate_limit=True)

In [0]:
domestic_india = pd.read_csv("domestic_india.csv")

In [0]:
domestic_india_sampled = domestic_india.sample(n=20000,random_state=88)

In [0]:
domestic_india_sampled1 = domestic_india_sampled.iloc[:5000]
domestic_india_sampled2 = domestic_india_sampled.iloc[5000:10000]
domestic_india_sampled3 = domestic_india_sampled.iloc[10000:15000]
domestic_india_sampled4 = domestic_india_sampled.iloc[15000:20001]

In [0]:
def get_location(username):
  try:
      userObj = api.get_user(username)
      print(userObj.location)
      return userObj.location
  except:
      return "No location"

In [0]:
from google.colab import files

In [21]:
domestic_india_sampled1['user_location'] = domestic_india_sampled1['username'].map(lambda u:get_location(u))
domestic_india_sampled1.to_csv("domestic_india_sampled1.csv",index=False)
files.download("domestic_india_sampled1.csv")
domestic_india_sampled2['user_location'] = domestic_india_sampled2['username'].map(lambda u:get_location(u))
domestic_india_sampled2.to_csv("domestic_india_sampled2.csv",index=False)
files.download("domestic_india_sampled2.csv")
domestic_india_sampled3['user_location'] = domestic_india_sampled3['username'].map(lambda u:get_location(u))
domestic_india_sampled3.to_csv("domestic_india_sampled3.csv",index=False)
files.download("domestic_india_sampled3.csv")
domestic_india_sampled4['user_location'] = domestic_india_sampled4['username'].map(lambda u:get_location(u))
domestic_india_sampled4.to_csv("domestic_india_sampled4.csv",index=False)
files.download("domestic_india_sampled4.csv")

CA
India
Maldives
Santa Rosa
New Delhi, India
Rohini, New Delhi
ᵘⁿⁱᵗᵉᵈ ᵃʳᵃᵇ ᵉᵐⁱʳᵃᵗᵉˢ🇦🇪
Malaysia
General Santos City
Selangor, Malaysia
Mogae 
pgh
New Delhi, India
Mumbai

Makai
Dammam
Asia
Philippines
Yes
Dammam
Kolkata, India
multi
Paranaque City, National Capit
New Delhi, India
Hong Kong
Chennai, India
Hickory, IN
Kuching, Sarawak
Victoria
her happiness 
Patna, India
Karachi, Pakistan
Dubai-UAE
New Delhi
sonepat, haryana
India
Pakistan
India
Gulbarga, India
Kuala Lumpur, Malaysia
Wellington City, New Zealand 
United Arab Emirates
68000
Lahore, Pakistan
Hyderabad, Bharat
Perak, Malaysia
Manila, Philippines
Malaysia
Indonesia
India
India
MNL
Dubai 
Hawaii
Pune, India
Kuwait
New Delhi, India
Agra, India
Dubai United Arab Emirates 
Pune, India
El Salvador
Pulwama, Jammu And Kashmir
Seoul, Republic of Korea
hawaii
MNL, PH
Indonesia
Jeddah, Kingdom of Saudi Arabia
jail :( ♬
India
8 

St. Louis Metro
Spain/China
china
most likely in bed
Shit City
Bukit Jelutong , Shah Alam
multi 
National C

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


MessageError: ignored

In [0]:
df1 = pd.read_csv("domestic_india_sampled1.csv")
df2 = pd.read_csv("domestic_india_sampled2.csv")
df3 = pd.read_csv("domestic_india_sampled3.csv")
df4 = pd.read_csv("domestic_india_sampled4.csv")

In [0]:
main_df = pd.concat([df1,df2,df3,df4],axis=0)

In [26]:
main_df.shape

(20000, 12)

In [0]:
main_df.drop_duplicates(keep="first",inplace=True)

In [28]:
main_df.shape

(19561, 12)

In [0]:
main_df.to_csv("domestic_india_states.csv",index=False)

In [0]:
tamilnadu = pd.read_csv("domestic_tn_states.csv")

In [33]:
tamilnadu.shape

(3766, 12)

In [0]:
tamilnadu.drop_duplicates(keep="first",inplace=True)

In [35]:
tamilnadu.shape

(2964, 12)

In [0]:
tamilnadu.to_csv("domestic_tamil_words.csv",index=False)